In [18]:
import cv2
import matplotlib.pyplot as plt

In [42]:
images = ['../Data/528.jpeg', '../Data/1084.jpg', '../Data/12146_И6125.jpg', '../Data/1753.jpeg', '../Data/12245_И6774.jpg',
          '../Data/12145_И6666.jpg', '../Data/1038.jpg','../Data/12173_И4242.jpg', '../Data/20250805_101846.jpg',
         '../Data/12156_И4559.jpg', '../Data/1038.jpg']


image = images[4]


template_path  = './ruler_template-h.png'

template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)


In [43]:
sift = cv2.ORB_create()

kp1, des1 = sift.detectAndCompute(template, None)
kp2, des2 = sift.detectAndCompute(img, None)

template1 = cv2.drawKeypoints(template, kp1, template)
img1 = cv2.drawKeypoints(img, kp2, img)


In [44]:

cv2.imshow("img", img1)
cv2.imshow("template", template1)

cv2.waitKey(0)
cv2.destroyAllWindows()
#plt.imshow(template)

In [35]:
%%time
import cv2
import numpy as np

# Load images
template = cv2.imread("ruler_template-h.png", cv2.IMREAD_GRAYSCALE)
scene = cv2.imread(image, cv2.IMREAD_FastFeatureDetector_createGRAYSCALE)

if template is None or scene is None:
    raise IOError("Could not load images")

# Create ORB detector
orb = cv2.ORB_create(
    nfeatures=2000,
    scaleFactor=1.2,
    nlevels=8
)

# Detect keypoints and compute descriptors
kp1, des1 = orb.detectAndCompute(template, None)
kp2, des2 = orb.detectAndCompute(scene, None)

# Create matcher (ORB uses binary descriptors → Hamming)
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

# Match descriptors using KNN
matches = bf.knnMatch(des1, des2, k=2)

# Apply Lowe's ratio test
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

print(f"Good matches: {len(good_matches)}")

# Draw matches
matched_img = cv2.drawMatches(
    template, kp1,
    scene, kp2,
    good_matches, None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

cv2.imshow("ORB Matches", matched_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ---- Optional: Locate template using homography ----
if len(good_matches) > 10:
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1,1,2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1,1,2)

    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    if H is not None:
        h, w = template.shape
        corners = np.float32([[0,0], [w,0], [w,h], [0,h]]).reshape(-1,1,2)
        projected = cv2.perspectiveTransform(corners, H)

        scene_color = cv2.cvtColor(scene, cv2.COLOR_GRAY2BGR)
        cv2.polylines(scene_color, [np.int32(projected)], True, (0,255,0), 3)

        #cv2.imshow("Detected Template", scene_color)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()


Good matches: 227
CPU times: user 331 ms, sys: 26.3 ms, total: 357 ms
Wall time: 2.12 s
